In [1]:
from carla import Benchmark
from IPython.display import display
import carla.evaluation.catalog as evaluation_catalog
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog
import shap 
from sklearn.preprocessing import OrdinalEncoder
from carla.data.causal_model import CausalModel
import shap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from carla.data.catalog import CsvCatalog
import numpy as np 

import warnings
warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2


/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jacqueline/.local/share/virtualenvs/CARLA-koH0yuP4/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint

[INFO] Using Python-MIP package version 1.12.0 [model.py <module>]


In [2]:
scm = CausalModel("adult")
dataset = scm.generate_dataset(10000)

display(dataset.df)

Adult Model Initiate
Structural Equation Finished
Noise Distribution Finished


,label,x1,x2,x3,x7,x8,x4,x5,x6,x9
0,1.0,0.002129,6.660761,42.976816,0.998782,2.430008,2.542741,81.754502,49374.870220,1980.405634
1,1.0,0.254379,10.820065,78.684519,13.514921,9.480391,3.270747,92.583894,78448.292556,3144.274441
2,1.0,0.187248,26.511345,60.668168,9.858080,0.930122,3.261694,89.331648,20592.419737,829.957202
3,1.0,0.500424,30.591215,54.923552,10.544465,12.349379,2.685984,6.646954,63612.298149,2546.800850
4,1.0,0.140297,12.631557,49.381875,12.041632,10.361903,1.290725,88.212093,99250.252830,3974.614694
...,...,...,...,...,...,...,...,...,...,...
9995,1.0,0.740898,28.946848,68.194456,2.565430,0.630566,0.388844,29.650945,35698.624892,1432.130569
9996,1.0,0.240870,23.853966,41.164084,4.308422,7.582912,0.097526,52.400976,98507.065123,3943.678927
9997,0.0,0.394263,23.438322,21.325679,2.319340,4.182362,3.334969,45.945921,2293.827062,94.205634
9998,1.0,0.139480,23.308218,33.605961,12.984307,0.390586,2.731525,85.551366,56267.244998,2254.872453


In [3]:
#Model 

training_params = {"lr": 0.01, "epochs": 3, "batch_size": 16, "hidden_size": [18, 9, 2]}

ml_model = MLModelCatalog(
    dataset, model_type="ann", load_online=False, backend="pytorch"
)
ml_model.train(
    learning_rate=training_params["lr"],
    epochs=training_params["epochs"],
    batch_size=training_params["batch_size"],
    hidden_size=training_params["hidden_size"],
    force_train=True
)


balance on test set 0.8649333333333333, balance on test set 0.8652
Epoch 0/2
----------
train Loss: 0.9822 Acc: 0.8593

test Loss: 0.3955 Acc: 0.8652

Epoch 1/2
----------
train Loss: 0.3965 Acc: 0.8649

test Loss: 0.3954 Acc: 0.8652

Epoch 2/2
----------
train Loss: 0.3964 Acc: 0.8649

test Loss: 0.3956 Acc: 0.8652



In [4]:
from carla.models.negative_instances import predict_negative_instances
import carla.recourse_methods.catalog as recourse_catalog

factuals = predict_negative_instances(ml_model, dataset.df)
test_factual = factuals.iloc[:20]

display(test_factual)

,label,x1,x2,x3,x7,x8,x4,x5,x6,x9


In [5]:
hyperparams = {"loss_type": "MSE", "binary_cat_features": False}

recourse_method = recourse_catalog.Wachter(ml_model, hyperparams)
cfs = recourse_method.get_counterfactuals(test_factual)
#from carla.models.negative_instances import predict_negative_instances
#from carla.recourse_methods.catalog.causal_recourse import (
#    CausalRecourse,
#    constraints,
#    samplers,
#)

#hyperparams = {
#    "optimization_approach": "brute_force",
#    "num_samples": 10,
#    "scm": scm,
#    "constraint_handle": constraints.point_constraint,
#    "sampler_handle": samplers.sample_true_m0,
#}
#cfs = CausalRecourse(ml_model, hyperparams).get_counterfactuals(test_factual)

display(cfs)

AttributeError: 'NoneType' object has no attribute 'get_feature_names'

In [ ]:
def get_pred_from_causal(scm,values,cf_label, mapping_dict):
    #TODO change everything hard coded
    values['target']=cf_label
    def _get_noise_string(node):
        def _get_node_id(node):
            return node[1:]
        if not node[0] == "x":
            raise ValueError
        return "u" + _get_node_id(node)
    exogenous_variables = np.concatenate(
        [
            np.array(values[mapping_dict[node]]).reshape(-1, 1)
            for node in scm.get_topological_ordering("exogenous")
        ],
        axis=1,
    )
    exogenous_variables = pd.DataFrame(
        exogenous_variables, columns=scm.get_topological_ordering("exogenous")
    )

    endogenous_variables = exogenous_variables.copy()
    endogenous_variables = endogenous_variables.rename(
        columns=dict(
            zip(
                scm.get_topological_ordering("exogenous"),
                scm.get_topological_ordering("endogenous"),
            )
        )
    )
    # used later to make sure parents are populated when computing children
    endogenous_variables.loc[:] = np.nan
    for node in scm.get_topological_ordering("endogenous"):
        parents = scm.get_parents(node)
        if endogenous_variables.loc[:, list(parents)].isnull().values.any():
            raise ValueError(
                "parents in endogenous_variables should already be occupied"
            )
        endogenous_variables[node] = scm.structural_equations_np[node](
            exogenous_variables[_get_noise_string(node)],
            *[endogenous_variables[p] for p in parents],
        )
    labels=endogenous_variables['x9'][0]
    return labels

In [ ]:
from carla.data import causal_model
from carla.evaluation import remove_nans
from carla.evaluation.api import Evaluation
class Sematic(Evaluation):
    """
    """

    def __init__(self, ml_model, causal_graph):
        self.ml_model= ml_model
        self.causal_graph=causal_graph
    def get_evaluation(self,factual: np.ndarray, counterfactual: np.ndarray, causal_model):
        # generate data 
        cf_label=self.ml_model.predict(np.array(counterfactual.values).reshape(1,-1))
        cf_label=np.argmax(cf_label)
        causal_label=get_pred_from_causal(self.causal_graph,counterfactual,cf_label, mapping_dict)
        print(cf_label)
        if cf_label ==causal_label:
            return 1 
        else: 
            return 0

class path_checker():
    pass


metric = Sematic(ml_model,scm)
result=metric.get_evaluation(np.array([]),df_cfs.iloc[0],mapping_dict)
''' 
Analysis
Probitlity Distributions 
Histogram True Positive, False Positives 
'''
'''
Parts of counterfactuals actually change 
1hop and 2 hop relations 
'''
'''Histogram Features / true Positive / false Positive'''
# first initialize the benchmarking class by passing
# black-box-model, recourse method, and factuals into it
#benchmark = Benchmark(ml_model, recourse_method, factuals)

# now you can decide if you want to run all measurements
# or just specific ones.
#evaluation_measures = [
#    evaluation_catalog.YNN(benchmark.mlmodel, {"y": 5, "cf_label": 1}),
#    evaluation_catalog.Distance(benchmark.mlmodel),
#    evaluation_catalog.SuccessRate(),
#    evaluation_catalog.Redundancy(benchmark.mlmodel, {"cf_label": 1}),
#    evaluation_catalog.ConstraintViolation(benchmark.mlmodel),
#    evaluation_catalog.AvgTime({"time": benchmark.timer}),
#]

# now run all implemented measurements and create a
# DataFrame which consists of all results
#results = benchmark.run_benchmark(evaluation_measures)


NameError: name 'df_cfs' is not defined